In [3]:
from func_search import *
from func_taux_transm import *
from func_def_class import *
from func_res import *
from func_parsing_logfile import *
from func_parsing_csvfile import *


In [4]:
import os
#import matplotlib.pyplot as plt
import re
import numpy as np
from path import Path
import csv
from pandas import *
import pandas as pd
import sys
import ast

In [5]:
#### PARSING DES LOGS DANS DES SOUS FICHIER TXT
sep_data_addr("data/exp_10.log")
sep_data_event("data/exp_10.log")

In [6]:
#### CREATION DES STRUCT PANDAS
#fichier tx
create_df_tx("data/parsed/event/data_STAT_PK_TX.log")


In [7]:
# fichier rx
create_df_rx("data/parsed/event/data_STAT_PK_RX.log", "data/parsed/event/data_STAT_ACK_RX.log")

In [8]:
df_tx = pd.read_csv(r'data_csv/pkt_tx.csv', dtype=str)
df_tx["asn"] = df_tx["asn"].astype(int)
df_tx["numTxAttempts"] = df_tx["numTxAttempts"].astype(int)
df_tx["trackinstance"] = df_tx["trackinstance"].astype(int)
df_tx = df_tx.fillna('')

df_rx = pd.read_csv(r'data_csv/pkt_rx.csv', dtype=str)
df_rx["asn"] = df_rx["asn"].astype(int)




In [9]:
#creer la df qui contient les infos sur les res
create_df_step_res("data/parsed/event/data_OTHER_ParserPrintf.log")

In [10]:
# fichiers res
create_df_res("data/parsed/event/data_OTHER_ParserPrintf.log")

In [11]:
df_res = pd.read_csv(r'data_csv/res.csv', dtype=str)
df_res["numAttempts"] = df_res["numAttempts"].astype(int)
#df_res["nb_sibl"] = df_res["nb_sibl"].astype(int)
df_res["succes"] = df_res["succes"].astype(int)
df_res["state"] = df_res["state"].astype(int)
df_res["asn creation"] = df_res["asn creation"].astype(int)
df_res = df_res.fillna('')
df_res.sort_values(['asn creation', 'asn_req'], ascending=[True, True], inplace=True)

In [12]:
#complete le df tx
fill_succes_tx(df_tx,df_rx)

In [13]:
#créer les fichiers parent et fils 
create_df_parent("data/parsed/event/data_OTHER_ParserPrintf.log")

In [14]:
df_parent = pd.read_csv(r'data_csv/parent.csv', dtype=str)
df_parent["asn"] = df_parent["asn"].astype(int)
df_parent = df_parent.fillna('')


In [15]:
create_df_sons(df_parent)

In [16]:
df_fils = pd.read_csv(r'data_csv/sons.csv', dtype=str)
df_fils["asn"] = df_fils["asn"].astype(int)


In [17]:
fill_nb_siblings(df_res, df_fils ,df_parent)

In [18]:
# a lancer après les siblings et qu'UNE fois 
fill_simult(df_res, df_tx)

In [19]:
df_res = pd.read_csv(r'data_csv/res.csv', dtype=str)
df_res["numAttempts"] = df_res["numAttempts"].astype(int)
df_res["nb_sibl"] = df_res["nb_sibl"].astype(int)
df_res["succes"] = df_res["succes"].astype(int)
df_res["state"] = df_res["state"].astype(int)
df_res["asn creation"] = df_res["asn creation"].astype(int)
df_res = df_res.fillna('')
df_res.sort_values(['asn creation', 'asn_req'], ascending=[True, True], inplace=True)

In [20]:
df_res.sort_values(['asn creation', 'asn_req', 'asn_tx'], ascending=[True, True, True], inplace=True)
df_res

,asn creation,asn_req,asn_rep,asn_tx,src,dest,owner,succes,numAttempts,colision,...,slot2,ch2,s2,slot3,ch3,s3,state,nb_sibl,nb_fils,diff_asn
0,1221,1299,1583,,b381,b868,b868,1,1,,...,163,12,0,93,11,1,4,2,,
99,1221,1299,1583,1258,b381,b868,b868,1,1,3,...,163,12,0,93,11,1,4,2,,
100,1221,1299,1583,1299,b381,b868,b868,1,1,1,...,163,12,0,93,11,1,4,2,,
226,1221,1299,1583,1542,b381,b868,b868,1,1,1,...,163,12,0,93,11,1,4,2,,
227,1221,1299,1583,1583,b381,b868,b868,1,1,1,...,163,12,0,93,11,1,4,2,,
1,1221,1339,1562,,8772,b868,b868,1,1,,...,38,14,0,141,9,1,4,2,,
97,1221,1339,1562,1258,8772,b868,b868,1,1,3,...,38,14,0,141,9,1,4,2,,
98,1221,1339,1562,1339,8772,b868,b868,1,1,1,...,38,14,0,141,9,1,4,2,,
225,1221,1339,1562,1562,8772,b868,b868,1,1,1,...,38,14,0,141,9,1,4,2,,
2,1221,2407,,,1162,b868,b868,0,1,,...,64,14,0,89,7,0,-1,2,,
